# **Mexico Toy Sales**


In [1]:
import pandas as pd
import numpy as np
import math
import os

calendar = pd.read_csv('./data/calendar.csv')
inventory = pd.read_csv('./data/inventory.csv')
products = pd.read_csv('./data/products.csv')
sales = pd.read_csv('./data/sales.csv')
stores = pd.read_csv('./data/stores.csv')

### **Tasks Overview**

Monthly summaries

- Last 2 years monthly average/max sales
- Last 3 months monthly average/max sales
- Same month last year sales
- Last month sales
- Current month sales

Weekly summaries

- Last 2 years weekly average/max sales
- Last 4 weeks weekly average/max sales
- Same week last year sales
- Last week sales
- Current week sales

Current Inventory


## **Data Manipulation**


In [2]:
sales.columns = sales.columns.str.lower()

# adding date, year, month and week values to the sales dataframe

sales['date'] = pd.to_datetime(sales['date'])
sales['year'] = sales['date'].dt.year
sales['month'] = sales['date'].values.astype('<M8[M]')
sales['week'] = sales['date'].dt.isocalendar().week

# we will combine the year and week column to create a new column week-id which will help
# late on to filter and sort the data

sales['week_id'] = sales['year'].astype(
    'str') + "-" + sales['week'].astype('str')

In [3]:
# with our previous week-id implementation, we may have instances with 2023-52 for a date such
# as 01-Januray-2023. We will fix this so that if the week > 50 while it is the first month
# then the week-id generated will have the value for year one less, like in this case 2022-52

sales['week_id'] = np.where(sales['month'].dt.month == 1,
                            np.where(sales['week'] > 50,
                                    (sales['year'] - 1).astype('str') + "-" + sales['week'].astype('str'),
                                    sales['week_id']),
                            sales['week_id']
                            )

# another problem that we have is that we have different length of strings such as 2023-1 and 2023-10
# this arises the short circuiting problem for example, if we were to filter out all the values for 
# week-id greater than 2023-30, then instead of having 2023-31, 2023-32, ... we instead will have
# 2023-4, 2023-5, ..., 2023-9, 2023-31, 2023-32 returning as a result.
# therefore we need to have the week-id consistent i.e., 2023-01, 2023-02, ..., 2023-09, 2023-10, ...

sales['week_id'] = np.where(sales['week_id'].str.len() == 6,
                            sales['week_id'].apply(
                                lambda x: f'{x[:5]}0{x[-1]}'),
                            sales['week_id']
                            )

In [4]:
# previewing dataframe

sales.head(5)

,sale_id,date,store_id,product_id,units,year,month,week,week_id
0,1,2022-01-01,24,4,1,2022,2022-01-01,52,2021-52
1,2,2022-01-01,28,1,1,2022,2022-01-01,52,2021-52
2,3,2022-01-01,6,8,1,2022,2022-01-01,52,2021-52
3,4,2022-01-01,48,7,1,2022,2022-01-01,52,2021-52
4,5,2022-01-01,44,18,1,2022,2022-01-01,52,2021-52


In [5]:
# constant values that we will be needing afterwards in the filters

CURRENT_DATE = sales['date'].max()
CURRENT_MONTH_START = CURRENT_DATE.replace(day=1)
CURRENT_WEEK_ID = f"{CURRENT_DATE.year}-{CURRENT_DATE.isocalendar().week}"


LAST_MONTH_START = CURRENT_MONTH_START - pd.DateOffset(months=1)
LAST_WEEK_ID = f"{(CURRENT_DATE - pd.DateOffset(weeks=1)).year}-{(CURRENT_DATE - pd.DateOffset(weeks=1)).isocalendar().week}"


LAST_YEAR_OFFSET_DATE = (
    CURRENT_DATE - pd.DateOffset(months=12)).replace(day=1)
LAST_YEAR_OFFSET_WEEK_ID = f"{LAST_YEAR_OFFSET_DATE.year}-{LAST_YEAR_OFFSET_DATE.isocalendar().week}"


TWO_YEARS_OFFSET_DATE = (
    CURRENT_DATE - pd.DateOffset(months=24)).replace(day=1)
TWO_YEARS_OFFSET_WEEK_ID = f"{TWO_YEARS_OFFSET_DATE.year}-{TWO_YEARS_OFFSET_DATE.isocalendar().week}"
THREE_MONTHS_OFFSET_DATE = (
    CURRENT_DATE - pd.DateOffset(months=3)).replace(day=1)
FOUR_WEEKS_OFFSET_ID = f"{(CURRENT_DATE - pd.DateOffset(weeks=4)).year}-{(CURRENT_DATE - pd.DateOffset(weeks=4)).isocalendar().week}"

### **Monthly Aggregations**


**Average/Max monthly sales in the last 2 year period**


In [6]:
# returns values aggregated over monthly aggregates for the last two year period (excluding current month)

last_two_years_monthly = (
    sales
    .loc[(sales.date >= TWO_YEARS_OFFSET_DATE) & (sales.month != CURRENT_MONTH_START)]
    .groupby(['product_id', 'store_id', 'month'])
    ['units']
    .sum()
    .groupby(['product_id', 'store_id'])
    .agg([
        ('max_units', 'max'),
        ('avg_units', lambda x: math.ceil(x.mean()))
    ])
    .reset_index()
)

last_two_years_monthly

,product_id,store_id,max_units,avg_units
0,1,1,99,61
1,1,2,120,34
2,1,3,112,57
3,1,4,86,50
4,1,5,143,44
...,...,...,...,...
1621,35,41,56,25
1622,35,42,20,9
1623,35,48,19,9
1624,35,49,31,16


**Average monthly sales in the last 3 months**


In [7]:
# returns monthly aggregated values for the last three months period (and fourth one as current month)

last_three_months_trend = (
    sales[['product_id', 'store_id']]
    .drop_duplicates()
    .merge(
        sales
        .loc[sales.date >= THREE_MONTHS_OFFSET_DATE]['month']
        .drop_duplicates()
        .reset_index(drop=True),
        how='cross'
    )
    .merge(
        sales
        .loc[(sales.date >= THREE_MONTHS_OFFSET_DATE)]
        .groupby(['product_id', 'store_id', 'month'])
        ['units']
        .sum()
        .reset_index(),
        on=['product_id', 'store_id', 'month'],
        how='left'
    )
    .sort_values(['product_id', 'store_id', 'month'])
    .reset_index(drop=True)
    .fillna(value={'units': 0})
)

last_three_months_trend

,product_id,store_id,month,units
0,1,1,2023-06-01,26.0
1,1,1,2023-07-01,45.0
2,1,1,2023-08-01,7.0
3,1,1,2023-09-01,57.0
4,1,2,2023-06-01,29.0
...,...,...,...,...
6519,35,49,2023-09-01,0.0
6520,35,50,2023-06-01,0.0
6521,35,50,2023-07-01,1.0
6522,35,50,2023-08-01,0.0


In [8]:
# returns values aggregated over monthly aggregates for the last three month period (excluding current month)

last_three_months_monthly = (
    last_three_months_trend
    .loc[last_three_months_trend.month != CURRENT_MONTH_START]
    .groupby(['product_id', 'store_id'])
    ['units']
    .agg([
        ('avg_units', lambda x: math.ceil(x.mean()))
    ])
    .reset_index()
)

last_three_months_monthly

,product_id,store_id,avg_units
0,1,1,26
1,1,2,31
2,1,3,25
3,1,4,36
4,1,5,26
...,...,...,...
1626,35,41,28
1627,35,42,8
1628,35,48,10
1629,35,49,4


**Same month last year sales**


In [9]:
same_month_last_year = (
    sales
    .loc[sales.month == LAST_YEAR_OFFSET_DATE]
    .groupby(['product_id', 'store_id'])
    ['units']
    .sum()
    .reset_index()
)

same_month_last_year

,product_id,store_id,units
0,1,1,49
1,1,2,10
2,1,3,32
3,1,4,14
4,1,5,13
...,...,...,...
1009,35,4,11
1010,35,7,9
1011,35,10,18
1012,35,23,6


**Last month sales**


In [10]:
last_month_sales = (
    sales
    .loc[sales.month == LAST_MONTH_START]
    .groupby(['product_id', 'store_id'])
    ['units']
    .sum()
    .reset_index()
)

last_month_sales

,product_id,store_id,units
0,1,1,7
1,1,2,9
2,1,3,10
3,1,4,31
4,1,5,18
...,...,...,...
1291,35,26,4
1292,35,37,17
1293,35,41,6
1294,35,48,17


**Current month sales**


In [11]:
current_month_sales = (
    sales
    .loc[sales.month == CURRENT_MONTH_START]
    .groupby(['product_id', 'store_id'])
    ['units']
    .sum()
    .reset_index()
)

current_month_sales

,product_id,store_id,units
0,1,1,57
1,1,2,67
2,1,3,34
3,1,4,29
4,1,5,13
...,...,...,...
1227,35,26,14
1228,35,38,24
1229,35,41,6
1230,35,48,17


### **Weekly Aggregations**


In [12]:
# storing the weekly sales data in a new dataframe

weekly_sales = (
    sales
    .loc[
        (sales.week_id >= TWO_YEARS_OFFSET_WEEK_ID) & (
            sales.week_id != CURRENT_WEEK_ID)
    ]
    .groupby(['product_id', 'store_id', 'year', 'week_id'])
    [['units']]
    .sum()
    .reset_index()
)

weekly_sales

,product_id,store_id,year,week_id,units
0,1,1,2022,2021-52,9
1,1,1,2022,2022-01,9
2,1,1,2022,2022-02,3
3,1,1,2022,2022-03,17
4,1,1,2022,2022-04,25
...,...,...,...,...,...
55385,35,50,2023,2023-06,2
55386,35,50,2023,2023-08,13
55387,35,50,2023,2023-14,19
55388,35,50,2023,2023-27,1


**Average/Max weekly sales in the last two year period**


In [13]:
# returns values aggregated over weekly aggregates for the last two year period (excluding current week)

last_two_years_weekly = (
    weekly_sales
    .groupby(['product_id', 'store_id'])
    ['units']
    .agg([
        ('max_units', 'max'),
        ('avg_units', lambda x: math.ceil(x.mean()))
    ])
    .reset_index()
)

last_two_years_weekly

,product_id,store_id,max_units,avg_units
0,1,1,66,19
1,1,2,71,14
2,1,3,49,19
3,1,4,66,18
4,1,5,53,18
...,...,...,...,...
1626,35,41,35,14
1627,35,42,18,8
1628,35,48,18,9
1629,35,49,22,11


**Average weekly sales Last in last four week**


In [14]:
# # don't need this, found a better way to filter the values

# def week_id_filter(current_date):

#     week = current_date.isocalendar().week
#     year = current_date.year

#     if week > 4:
#         return [f'{year}-{i}' for i in range(week-4, week)]

#     if week == 1:
#         return [f'{year-1}-{i}' for i in range(48+week, 53)]

#     tmp = [f'{year-1}-{i}' for i in range(48+week, 53)]
#     tmp.extend(f'{year}-{i}' for i in range(1, week))

#     return tmp

In [15]:
# returns weekly aggregates for the last four week period (fifth one as the current week)

last_four_weeks_trend = (
    sales[['product_id', 'store_id']]
    .drop_duplicates()
    .merge(
        sales
        .loc[sales.week_id >= FOUR_WEEKS_OFFSET_ID]['week_id']
        .drop_duplicates()
        .reset_index(drop=True),
        how='cross'
    )
    .merge(
        sales
        .loc[sales.week_id >= FOUR_WEEKS_OFFSET_ID]
        .groupby(['product_id', 'store_id', 'week_id'])
        ['units']
        .sum()
        .reset_index(),
        on=['product_id', 'store_id', 'week_id'],
        how='left'
    )
    .sort_values(['product_id', 'store_id', 'week_id'])
    .reset_index(drop=True)
    .fillna(value={'units': 0})
)

last_four_weeks_trend

,product_id,store_id,week_id,units
0,1,1,2023-35,29.0
1,1,1,2023-36,0.0
2,1,1,2023-37,0.0
3,1,1,2023-38,15.0
4,1,1,2023-39,17.0
...,...,...,...,...
8150,35,50,2023-35,0.0
8151,35,50,2023-36,0.0
8152,35,50,2023-37,0.0
8153,35,50,2023-38,3.0


In [16]:
# returns values aggregated over weekly aggregates for the last four weeks period (excluding current week)

last_four_weeks_weekly = (
    last_four_weeks_trend
    .loc[last_four_weeks_trend.week_id != CURRENT_WEEK_ID]
    .groupby(['product_id', 'store_id'])
    ['units']
    .agg(
        [
            ('avg_units', lambda x: math.ceil(x.mean()))
        ]
    )
    .reset_index()
)

last_four_weeks_weekly

,product_id,store_id,avg_units
0,1,1,11
1,1,2,11
2,1,3,9
3,1,4,6
4,1,5,5
...,...,...,...
1626,35,41,0
1627,35,42,0
1628,35,48,0
1629,35,49,0


**Same week last year sales**


In [17]:
same_week_last_year = (
    weekly_sales
    .loc[weekly_sales.week_id == LAST_YEAR_OFFSET_WEEK_ID]
    .groupby(['product_id', 'store_id'])
    ['units']
    .sum()
    .reset_index()
)

same_week_last_year

,product_id,store_id,units
0,1,5,13
1,1,9,47
2,1,11,28
3,1,14,27
4,1,15,14
...,...,...,...
523,34,42,11
524,34,46,3
525,34,47,20
526,35,4,11


**Last week sales**


In [18]:
last_week_sales = (
    last_four_weeks_trend
    .loc[last_four_weeks_trend.week_id == LAST_WEEK_ID]
    [['product_id', 'store_id', 'units']]
)

last_week_sales

,product_id,store_id,units
3,1,1,15.0
8,1,2,7.0
13,1,3,0.0
18,1,4,10.0
23,1,5,4.0
...,...,...,...
8133,35,41,0.0
8138,35,42,0.0
8143,35,48,0.0
8148,35,49,0.0


**Current week sales**


In [19]:
current_week_sales = (
    last_four_weeks_trend
    .loc[last_four_weeks_trend.week_id == CURRENT_WEEK_ID]
    [['product_id', 'store_id', 'units']]
)

current_week_sales

,product_id,store_id,units
4,1,1,17.0
9,1,2,23.0
14,1,3,11.0
19,1,4,8.0
24,1,5,0.0
...,...,...,...
8134,35,41,6.0
8139,35,42,0.0
8144,35,48,17.0
8149,35,49,0.0


## **Exporting files**


In [20]:
combined_data = (
    sales[['product_id', 'store_id']]
    .drop_duplicates()
    .sort_values(['product_id', 'store_id'])
    .reset_index(drop=True)
)

In [21]:
# merging all the dataframes to create on gargantuan one

combined_data = (
    combined_data

    .merge(
        last_two_years_monthly
        .rename(columns={
                'max_units': 'ty_max_month',
                'avg_units': 'ty_avg_month'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        last_three_months_monthly
        .rename(columns={
                'avg_units': 'tm_avg'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        same_month_last_year
        .rename(columns={
                'units': 'ly_same_month'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        last_month_sales
        .rename(columns={
                'units': 'last_month'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        current_month_sales
        .rename(columns={
                'units': 'current_month'
                }),
        on=['product_id', 'store_id'],
        how='left')
)

In [22]:
combined_data = (
    combined_data

    .merge(
        last_two_years_weekly
        .rename(columns={
                'avg_units': 'ty_avg_week',
                'max_units': 'ty_max_week'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        last_four_weeks_weekly
        .rename(columns={
                'avg_units': 'fw_avg'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        same_week_last_year
        .rename(columns={
                'units': 'ly_same_week'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        last_week_sales
        .rename(columns={
                'units': 'last_week'
                }),
        on=['product_id', 'store_id'],
        how='left')

    .merge(
        current_week_sales
        .rename(columns={
                'units': 'current_week'
                }),
        on=['product_id', 'store_id'],
        how='left')
)

In [23]:
combined_data.head(5)

,product_id,store_id,ty_max_month,ty_avg_month,tm_avg,ly_same_month,last_month,current_month,ty_max_week,ty_avg_week,fw_avg,ly_same_week,last_week,current_week
0,1,1,99.0,61.0,26,49.0,7.0,57.0,66,19,11,NaN,15.0,17.0
1,1,2,120.0,34.0,31,10.0,9.0,67.0,71,14,11,NaN,7.0,23.0
2,1,3,112.0,57.0,25,32.0,10.0,34.0,49,19,9,NaN,0.0,11.0
3,1,4,86.0,50.0,36,14.0,31.0,29.0,66,18,6,NaN,10.0,8.0
4,1,5,143.0,44.0,26,13.0,18.0,13.0,53,18,5,13.0,4.0,0.0


In [24]:
combined_data = (
    combined_data
    .merge(
        inventory,
        left_on=['product_id', 'store_id'],
        right_on=['Product_ID', 'Store_ID'],
        how='left'
    )
    .rename(columns={'Stock_On_Hand': 'current_stock'})
    .drop(['Product_ID', 'Store_ID'], axis=1)
    .fillna(0)
)

In [25]:
# writing files to disk

try: os.mkdir('./clean-data')
except: pass

combined_data.to_csv('./clean-data/combined_data.csv', index=False)
last_three_months_trend.to_csv(
    './clean-data/three_months_trend.csv', index=False)
last_four_weeks_trend.to_csv(
    './clean-data/four_weeks_trend.csv', index=False)